In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
import pandas as pd
file_path = 'pre_processed_data.csv'
df = pd.read_csv(file_path)


In [20]:
def map_sentiment(rating):
    if rating >= 4.0:
        return 'positive'
    elif rating <= 2.0:
        return 'negative'
    else:
        return 'neutral'

df['sentiment'] = df['rating'].apply(map_sentiment)


In [21]:

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

X = df['content']
y = df['sentiment']

# Map sentiment labels to numeric values
label_mapping = {'positive': 2, 'neutral': 1, 'negative': 0}
y = y.map(label_mapping)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize and convert text data to sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

max_sequence_length = 100
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post')

# Define model architecture
embedding_dim = 50
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=max_sequence_length),
    LSTM(units=128),
    Dense(units=3, activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train_padded, y_train, epochs=10, batch_size=64, validation_split=0.1)

# Evaluate model on test data
loss, accuracy = model.evaluate(X_test_padded, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/10
25/25 [==============================] - 2s 27ms/step - loss: 0.5640 - accuracy: 0.8778 - val_loss: 0.4234 - val_accuracy: 0.8870
Epoch 2/10
25/25 [==============================] - 0s 16ms/step - loss: 0.3493 - accuracy: 0.9150 - val_loss: 0.4218 - val_accuracy: 0.8870
Epoch 3/10
25/25 [==============================] - 0s 16ms/step - loss: 0.3435 - accuracy: 0.9150 - val_loss: 0.4428 - val_accuracy: 0.8870
Epoch 4/10
25/25 [==============================] - 0s 14ms/step - loss: 0.3482 - accuracy: 0.9150 - val_loss: 0.4299 - val_accuracy: 0.8870
Epoch 5/10
25/25 [==============================] - 0s 14ms/step - loss: 0.3460 - accuracy: 0.9150 - val_loss: 0.4199 - val_accuracy: 0.8870
Epoch 6/10
25/25 [==============================] - 0s 16ms/step - loss: 0.3441 - accuracy: 0.9150 - val_loss: 0.4206 - val_accuracy: 0.8870
Epoch 7/10
25/25 [==============================] - 1s 20ms/step - loss: 0.3404 - accuracy: 0.9150 - val_loss: 0.4328 - val_accuracy: 0.8870
Epoch 8/10
25